# Introduction

This notebook will will help you create and train your own custom TensorFlowJS model for classifying a character in a 32x32 image.  

It's super simple - just collapse the "Code" section and click "Play"! And, of course, if you want to know the details, feel free to check out the code. I tried to keep it as simple as possible.

## Using Google Colab
If you want to train your network as fast as possible, run this notebook in Google Colab and go to `Runtime -> Change runtime type` and select `GPU`.

There's a catch though. Colab kicks you after 90 minutes of inactivity, so you can't leave the model training unattended. A simple workaround for this is to run this code in your browser console:
```js
const clickConnect = () => {
    var connectButton = document
        .querySelector('#top-toolbar > colab-connect-button')
        .shadowRoot.querySelector('#connect');
 
    connectButton.click();
    console.log(`Clicked at ${new Date()}`);
};
 
intervalId = setInterval(clickConnect, 600000);
```
This will click a button in Colab every 10 minutes. To stop it, use this:
```js
window.clearInterval(intervalId);
```

Remember though, Colab will still kick you after 12 hours of usage, so don't forget to save your files to an external source!

# Code

## Prepare workspace

### Define common variables

In [ ]:
IMAGE_SIZE = 32

EPOCHS = 300

BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
RANDOM_SEED = 0


### Install libraries

In [ ]:
!pip install tensorflowjs

In [ ]:
!apt install rar

### Fetch dataset

In [ ]:
# Download the dataset from GDrive
!gdown --id "1F98iX3a2AwOP7anep_ybUul-fuwfL6qM" --output characters.rar

In [ ]:
# Unrar it to ./characters/
!rm -r characters
!unrar x -idq characters.rar

## Create, train and export the model

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Create train and validation datasets
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'characters',
    validation_split=VALIDATION_SPLIT,
    subset='training',
    seed=RANDOM_SEED,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'characters',
    validation_split=VALIDATION_SPLIT,
    subset='validation',
    seed=RANDOM_SEED,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

classes = raw_train_ds.class_names
num_classes = len(classes)
print(f'Class names ({num_classes}):', classes)

# Normalize images (map pixel values from 0-255 to 0-1)
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
train_ds = raw_train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = raw_val_ds.map(lambda x, y: (normalization_layer(x), y))

# Create a simple sequential model
model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
)

# Once training is done, convert the model to a TensorFlowJS LayersModel and save it
print('Training finished, saving the model... ', end='')
tfjs.converters.save_keras_model(model, 'tfjs_model')
print('Done!')

# Plot training stats
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('training_graphs.png')



## Archive and download the saved TensorFlowJS model

In [ ]:
# Compress the exported model to tfjs_model.rar
!rar a tfjs_model.rar tfjs_model/

In [ ]:
from google.colab import files

# Download the compressed model
files.download('tfjs_model.rar')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>